In [2]:
import scipy.io as sio

mat_contents = sio.loadmat("hw1data.mat")
mat_contents

{'X': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 'Y': array([[9],
        [5],
        [9],
        ..., 
        [4],
        [0],
        [9]], dtype=uint8), '__globals__': [], '__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sat May 30 00:42:33 2015', '__version__': '1.0'}

In [3]:
import numpy as np
X = np.asarray(mat_contents['X'], dtype=np.int32) # Data Matrix
Y = np.asarray(mat_contents['Y'], dtype=np.int8).reshape(10000,1)# Labels


In [5]:
# Preprocessing data set
import random
np.random.shuffle(X)
rows = X.shape[0] * 0.8
training, trainingLabels = X[:rows,:], Y[:rows,:]
testing, testingLabels = X[rows:,:], Y[rows:,:]

C:\Users\lucas\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\lucas\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [11]:
### Selecting the top 200 useful features
import heapq
import numpy as np

def summarize(dataset):
    summaries = [(np.mean(attribute), np.std(attribute)) for attribute in zip(*dataset)]
    return summaries

summaries = summarize(X)
variances = np.empty(784)
for i in range(len(summaries)):
    variances[i] = summaries[i][1]
    
best_feature_indice = (variances).argsort()[-200:][::-1]
#print(best_feature_indice)

## After selecting the most valuable features
trainingSet = training[:, best_feature_indice]
print(trainingSet.shape)
testingSet = testing[:, best_feature_indice]

(8000, 200)


In [12]:
combined = np.append(trainingSet, trainingLabels, axis = 1)
print(combined)
# X = np.asarray(mat_contents['X'], dtype=np.int32) # Data Matrix
# Y = np.asarray(mat_contents['Y'], dtype=np.int8)# Labels

[[254 254   0 ..., 179   0   9]
 [  0   0   0 ..., 181   0   5]
 [  0   0   0 ..., 253   0   9]
 ..., 
 [  0   0 252 ...,   0   0   3]
 [  0   0 255 ...,   0  57   5]
 [  0   0 255 ...,   0 255   7]]


In [ ]:
# Calculate the Gini index for a split dataset, given the splited group (left and right) and class values
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    
#     for group in groups:
#         print (len(group)) 

    #print (n_instances)
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        #print("group:", group)
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            
            p = [row[-1] for row in group].count(class_val) / size
            # p is the proportion 
            
            score += p * p
            
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Select the best split point for a dataset
def get_split(dataset, labels):
    class_values = list(set(row[-1] for row in labels))
    
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    
    for index in range(len(dataset[0])-1):
        
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

def get_split(dataset):
    
    class_values = list(set(row[-1] for row in dataset))
    print("class values are ", class_values)
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            #print(groups) #groups are the left and right after Gini Index split
            #class values are the labels
            gini = gini_index(groups, class_values)
            #print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
                
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]
#split = get_split(X, Y)

sampledata = np.asarray(dataset)
samplelabels = sampledata[:, [-1]]
sampledata = sampledata[:, [0,1]]
#print(sampledata)
#split = get_split(sampledata, samplelabels)
split = get_split(combined)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
